In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import csv
from sklearn.linear_model import LinearRegression
#linreg gives straight line
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsRegressor
#doesnt work idk
from sklearn.ensemble import RandomForestClassifier
#doesnt work
from sklearn.ensemble import RandomForestRegressor
#gives me 1 entire value
from sklearn import tree
from sklearn.ensemble import GradientBoostingRegressor
#gives me 1 value
from sklearn.multioutput import MultiOutputRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import MultiTaskLasso
from sklearn.gaussian_process import GaussianProcessRegressor
import time

In [2]:
train = pd.read_csv("ais_train.csv", sep='|')
test = pd.read_csv("ais_test.csv")
train['time'] = pd.to_datetime(train['time'])
test['time'] = pd.to_datetime(test['time'])

test.tail()

,ID,vesselId,time,scaling_factor
51734,51734,61e9f3a8b937134a3c4bfdf3,2024-05-12 23:59:58,0.1
51735,51735,61e9f3b4b937134a3c4bfe77,2024-05-12 23:59:58,0.1
51736,51736,61e9f46cb937134a3c4c02b7,2024-05-12 23:59:58,0.1
51737,51737,61e9f465b937134a3c4c0269,2024-05-12 23:59:58,0.1
51738,51738,61e9f3adb937134a3c4bfe39,2024-05-12 23:59:58,0.1


In [3]:
def preprocess(df):
    data = df.copy()
    time3 = []
    for i in data['time']:
        time3.append(int((i.timestamp()-1704067200)/60))
    data['time'] = np.array(time3, dtype = 'float32')
    data = pd.get_dummies(data, columns=['vesselId'], drop_first=True)
    return(data)

test2 = test.copy()
df2 = train.copy()
goaldata = df2.drop(columns=['cog','sog','rot','heading','navstat','etaRaw','portId','time','vesselId'])
df2 = preprocess(df2)
trainingdata = df2.drop(columns=['cog','sog','rot','heading','navstat','etaRaw','latitude','longitude','portId'])
test2 = preprocess(test2)
test2 = test2.drop(columns=['ID','scaling_factor'])
for column in trainingdata:
    if column not in test2:
        test2[column] = False
test2 = test2.reindex(sorted(test2.columns), axis=1)
trainingdata = trainingdata.reindex(sorted(trainingdata.columns), axis=1)
test3 = test2[test2['vesselId_61e9f469b937134a3c4c029b']== True]
test3.head()

/tmp/ipykernel_8230/19055419.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test2[column] = False
/tmp/ipykernel_8230/19055419.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test2[column] = False
/tmp/ipykernel_8230/19055419.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test2[colum

,time,vesselId_61e9f38eb937134a3c4bfd8d,vesselId_61e9f38eb937134a3c4bfd8f,vesselId_61e9f38eb937134a3c4bfd91,vesselId_61e9f390b937134a3c4bfd93,vesselId_61e9f391b937134a3c4bfd95,vesselId_61e9f391b937134a3c4bfd97,vesselId_61e9f392b937134a3c4bfd99,vesselId_61e9f392b937134a3c4bfd9b,vesselId_61e9f393b937134a3c4bfd9d,...,vesselId_clh682fbq000qghxd3vn5xo01,vesselId_clh682fbq000rghxd2u75zdbn,vesselId_clh6aqawa0000gh0z7nc8e6nd,vesselId_clh6aqawa0001gh0zmijpuho1,vesselId_clh6aqawa0002gh0zypfa5dut,vesselId_clh6aqawa0003gh0zu0aznvt2,vesselId_clh6aqawa0004gh0z12aogec9,vesselId_clh6aqawa0005gh0z64y4xyip,vesselId_clh6aqawa0006gh0zje911dl3,vesselId_clh6aqawa0007gh0z9h6zi9bo
2,184330.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4620,185007.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4758,185033.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4975,185056.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5016,185072.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [5]:
model = LinearRegression()
start = time.time()
model.fit(trainingdata,goaldata)
end = time.time()
print(end - start)

34.886781215667725


In [19]:
start = time.time()
res = model.predict(test3)
end = time.time()
print(end - start)

0.02768540382385254


In [7]:
ramg = [i for i in range(51739)]
res2 = [ramg,[i[1] for i in res],[i[0] for i in res]]
with open('res.csv', 'w') as f:
    write = csv.writer(f)
    write.writerow(["ID","longitude_predicted","latitude_predicted"])
    write.writerows(zip(*res2))
df = pd.read_csv('res.csv')
df.to_csv('res.csv', index=False)

In [8]:
print(res)

[[39.47072    8.852467 ]
 [39.46735    8.850128 ]
 [39.46722    8.850039 ]
 [39.467106   8.849959 ]
 [39.467026   8.849904 ]
 [39.464775   8.848343 ]
 [39.45674    8.842768 ]
 [39.456642   8.842699 ]
 [39.456535   8.842627 ]
 [39.456444   8.842561 ]
 [39.456337   8.842488 ]
 [39.456264   8.842437 ]
 [39.45616    8.842367 ]
 [39.45606    8.842295 ]
 [39.455956   8.842222 ]
 [39.45585    8.84215  ]
 [39.45575    8.842081 ]
 [39.455647   8.842009 ]
 [39.455547   8.841939 ]
 [39.45544    8.8418665]
 [39.45534    8.841798 ]
 [39.455242   8.841728 ]
 [39.45517    8.841677 ]
 [39.45507    8.841608 ]
 [39.454998   8.841559 ]
 [39.454758   8.841393 ]
 [39.454754   8.841391 ]
 [39.45311    8.84025  ]
 [39.453003   8.840174 ]
 [39.452873   8.840084 ]
 [39.452774   8.840015 ]
 [39.45272    8.839977 ]
 [39.452576   8.839877 ]
 [39.45079    8.83864  ]
 [39.450756   8.838616 ]
 [39.450584   8.838495 ]
 [39.450485   8.838427 ]
 [39.45038    8.838354 ]
 [39.450283   8.838288 ]
 [39.448494   8.837045 ]
